In [1]:
from google.colab import drive
# Kết nối Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import matplotlib.pyplot as plt
from google.colab import drive
from keras.callbacks import EarlyStopping


In [3]:
# Kết nối Google Drive
drive.flush_and_unmount()  # Ngắt kết nối
drive.mount('/content/drive')  # Kết nối lại

# Đường dẫn tới thư mục chứa dữ liệu
base_dir = 'drive/MyDrive/CNN_MobileNetV2/jpg'
img_size = 224  # Đặt kích thước hình ảnh thành 224x224
batch = 64

# Tạo generator cho tập huấn luyện và kiểm tra với tăng cường dữ liệu
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,  # Thêm xoay
    width_shift_range=0.2,  # Di chuyển chiều rộng
    height_shift_range=0.2,  # Di chuyển chiều cao
    validation_split=0.2)  # 80% train, 20% val

test_datagen = ImageDataGenerator(rescale=1. / 255,
                                  validation_split=0.2)

# Tạo datasets
train_generator = train_datagen.flow_from_directory(base_dir,
                                                    target_size=(img_size, img_size),
                                                    subset='training',
                                                    batch_size=batch)

validation_generator = test_datagen.flow_from_directory(base_dir,
                                                        target_size=(img_size, img_size),
                                                        subset='validation',
                                                        batch_size=batch)

Mounted at /content/drive
Found 5935 images belonging to 102 classes.
Found 1435 images belonging to 102 classes.


In [4]:
# Hàm xây dựng mô hình CNN với kiến trúc MobileNetV2
def build_model(num_classes):
    inputs = layers.Input(shape=(img_size, img_size, 3))  # Kích thước đầu vào
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
    base_model.trainable = False  # Không huấn luyện lại mô hình cơ sở

    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs=base_model.input, outputs=outputs)
    return model

# Xây dựng và biên dịch mô hình
num_classes = len(train_generator.class_indices)
model = build_model(num_classes)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# model.fit(train_generator,epochs=20,validation_data=validation_generator)
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(train_generator, epochs=40, validation_data=validation_generator, callbacks=[early_stopping])

Epoch 1/40


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


93/93 ━━━━━━━━━━━━━━━━━━━━ 2364s 24s/step - accuracy: 0.0244 - loss: 4.8409 - val_accuracy: 0.1526 - val_loss: 4.1618
Epoch 2/40
93/93 ━━━━━━━━━━━━━━━━━━━━ 472s 5s/step - accuracy: 0.1147 - loss: 4.1753 - val_accuracy: 0.2690 - val_loss: 3.6349
Epoch 3/40
93/93 ━━━━━━━━━━━━━━━━━━━━ 476s 5s/step - accuracy: 0.2084 - loss: 3.6992 - val_accuracy: 0.3763 - val_loss: 3.1331
Epoch 4/40
93/93 ━━━━━━━━━━━━━━━━━━━━ 502s 5s/step - accuracy: 0.2880 - loss: 3.2683 - val_accuracy: 0.4502 - val_loss: 2.7207
Epoch 5/40
93/93 ━━━━━━━━━━━━━━━━━━━━ 505s 5s/step - accuracy: 0.3451 - loss: 2.9381 - val_accuracy: 0.5352 - val_loss: 2.3669
Epoch 6/40
93/93 ━━━━━━━━━━━━━━━━━━━━ 489s 5s/step - accuracy: 0.4135 - loss: 2.5677 - val_accuracy: 0.5847 - val_loss: 2.0784
Epoch 7/40
93/93 ━━━━━━━━━━━━━━━━━━━━ 542s 6s/step - accuracy: 0.4370 - loss: 2.3813 - val_accuracy: 0.6091 - val_loss: 1.8650
Epoch 8/40
93/93 ━━━━━━━━━━━━━━━━━━━━ 515s 5s/step - accuracy: 0.4787 - loss: 2.1693 - val_accuracy: 0.6599 - val_loss: 

In [ ]:
# In cấu trúc mô hình trước khi lưu
model.summary()

In [ ]:
# Lưu mô hình
model.save('drive/MyDrive/CNN_MobileNetV2/cnn_mobilenetv2_model.keras')

# Tải lại mô hình
loaded_model = tf.keras.models.load_model('drive/MyDrive/CNN_MobileNetV2/cnn_mobilenetv2_model.keras')

# Đánh giá mô hình trên tập dữ liệu kiểm tra
test_loss, test_accuracy = loaded_model.evaluate(validation_generator)
print(f'Test loss: {test_loss:.4f}, Test accuracy: {test_accuracy:.4f}')
loaded_model.summary()

In [ ]:

# Đánh giá mô hình
test_loss, test_acc = model.evaluate(validation_generator)
print(f'Test accuracy: {test_acc}')


In [ ]:

# Hiển thị đồ thị quá trình huấn luyện
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss')
plt.show()


In [ ]:
# Đánh giá mô hình
test_loss, test_acc = model.evaluate(validation_generator)
print(f'Test accuracy: {test_acc}')


In [ ]:
# Hiển thị đồ thị quá trình huấn luyện
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()

# Confusion matrix và báo cáo chi tiết


In [ ]:
class_counts = {v: k for k, v in train_generator.class_indices.items()}
print(class_counts)

In [ ]:
# Cập nhật hàm dự đoán và hiển thị hình ảnh
def predict_image(image_path, model, file_name):
    # Tải và hiển thị hình ảnh đầu vào
    img_size = 224  # Kích thước ảnh đầu vào cho mô hình
    test_image = image.load_img(image_path, target_size=(img_size, img_size))

    # Hiển thị hình ảnh
    plt.imshow(test_image)
    plt.axis('off')  # Ẩn trục
    plt.show()

    # Tiền xử lý hình ảnh
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    test_image = test_image / 255.0  # Chuẩn hóa hình ảnh

    # Dự đoán lớp
    result = model.predict(test_image)
    predicted_class_index = np.argmax(result[0])
    predicted_class = class_counts[predicted_class_index]  # Lấy tên lớp từ keys

    # In tên tệp và kết quả dự đoán
    print(f'Processing file: {file_name}')  # In tên tệp
    print(f'Predicted Class: {predicted_class}')

    # Làm tròn xác suất
    probabilities = np.round(result[0], 4)  # Làm tròn tới 4 chữ số thập phân

    # Định dạng xác suất để không có ký hiệu "e"
    formatted_probabilities = ["{:.4f}".format(prob) for prob in probabilities]

    # Tạo đầu ra cho Probabilities với nhãn lớp
    probabilities_output = [f"{label}: '{prob}'" for label, prob in zip(class_counts.values(), formatted_probabilities)]

    # In Probabilities theo định dạng mong muốn
    # In Probabilities theo định dạng mong muốn
    # In Probabilities theo định dạng mong muốn

    # Hiển thị nhãn theo kiểu bạn mong muốn
    labels = np.zeros_like(probabilities)
    labels[predicted_class_index] = 1

    # Tạo đầu ra cho labels
    labels_matrix = labels.reshape(1, -1)  # Chuyển thành ma trận 1 hàng
    print(labels_matrix)

    return predicted_class, result[0]  # Trả về cả lớp dự đoán và xác suất

# Lấy tất cả các tệp hình ảnh trong thư mục
image_dir = 'drive/MyDrive/TrainAI/test_classes_jpg/'  # Đường dẫn đến thư mục chứa hình ảnh cần dự đoán
image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Dự đoán cho tất cả các hình ảnh và in tên tệp
for image_file in image_files:
    image_path = os.path.join(image_dir, image_file)  # Đường dẫn đầy đủ đến tệp
    predicted_class, probabilities = predict_image(image_path, model, image_file)